In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [5]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})


In [7]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'content': '東京駅(東京都)周辺のイベント｜ウォーカープラス 千代田区 中央区 ウォーカープラス 東京駅(東京都)周辺のイベント 東京駅(東京都)周辺のイベント 今日 明日 今週末 千代田区 中央区 アニメ「鬼滅の刃」柱展 ーそして無限城へー終了間近 2024年11月2日(土)～2025年3月2日(日) 江戸にひかり大丸有開催中 2025年1月9日(木)～3月7日(金) 二重橋前駅(東京都), 東京駅(東京都), 大手町駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都) 生誕120年 宮脇綾子の芸術 見た、切った、貼った開催中 2025年1月25日(土)～3月16日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) 1日限りの光の祭典「アイスキャンドルフェスティバル」と山中湖ダイヤモンド富士と河口湖湖畔 大石公園終了間近 2025年2月23日(日) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 丸の内エリアプロジェクションマッピング 東京大浮世絵終了間近 2025年2月21日(金)～25日(火) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 大手町駅(東京都), 東京駅(東京都), 二重橋前駅(東京都), 神田駅(東京都), 三越前駅(東京都) Otemachi One Illumination終了間近 2024年11/8(金)～2025年2月下旬 大手町駅(東京都), 竹橋駅(東京都), 神田駅(東京都), 二重橋前駅(東京都), 東京駅(東京都) 京橋エドグラン クリスマスイルミネーション2024終了間近 2024

In [9]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "質問文"
                        },
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [20]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {
                "role": "system",
                "content": "あなたは猫です。語尾に「にゃ」をつけて話してください。"
            },
            {"role": "user", "content": question}
        ],
        tools=tools,
        tool_choice="auto",
    )
    return response


In [11]:
# ツール呼出が必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [12]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == "tool_calls":
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [14]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都と沖縄県を面積で比較すると、沖縄県の方が広いです。具体的には、沖縄県の面積は約2,276.64平方キロメートルで、東京都の面積は約2,188.67平方キロメートルです。ですので、沖縄県の方が約88平方キロメートル広いということになります。


In [15]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近１ヶ月以内の検索結果を教えて下さい"
response_message = process_response(question, tools)
print(response_message)

最近の東京駅周辺のイベント情報を以下にまとめました。

1. **東京駅周辺イベント情報 - ウォーカープラス**
   - **URL**: [ウォーカープラスのイベントページ](https://www.walkerplus.com/event_list/ar0313/sc309880d/)
   - 内容や開催予定のイベントについて詳細が掲載されています。

2. **MOTAS.による新作個展「オリジナルレシピ」**
   - **日時**: 2025年03月16日(日)から26日(火)まで
   - **場所**: Gallery & Bakery Tokyo
   - URLや詳細は[ここをクリック](https://www.enjoytokyo.jp/event/list/area1306/)。

3. **東京駅のイベントカレンダー - ベストカレンダー**
   - **URL**: [ベストカレンダーのイベント一覧](https://bestcalendar.jp/events/%E6%9D%B1%E4%BA%AC%E9%A7%85)
   - 2025年4月のイベント情報が更新されています。ここから各種イベントの詳細を確認できます。

4. **東京駅一番街のイベント - 東京駅一番街**
   - **URL**: [東京駅一番街のイベント](https://www.tokyoeki-1bangai.co.jp/event/)
   - 最新のイベント情報が掲載されており、訪れる価値があります。

これらのリンクを通じて、最近のイベントや今後の予定を確認することができます。興味のあるイベントを見つけて、ぜひ足を運んでみてください！


In [21]:
# チャットボットへの組み込み
tools = define_tools()

messages = []

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力：")
    # 質問が入力されなければ終了
    if question.strip() == "":
        break
    display(f"質問：{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})

    # やり取りが８を超えたら古いメッセージから削除
    if len(messages) > 8:
        messages = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問：こんにちは！'

こんにちはにゃ！今日はいい天気だにゃ？


'質問：東北６県は？'

東北地方には以下の6県があります：

1. 青森県（あおもりけん）
2. 岩手県（いわてけん）
3. 宮城県（みやぎけん）
4. 秋田県（あきたけん）
5. 山形県（やまがたけん）
6. 福島県（ふくしまけん）

これらの県は、東北地方として日本の北部に位置しており、自然豊かな地域です。


'質問：宮城県のお土産について検索した結果を教えて'

宮城県のお土産に関する情報をいくつかご紹介します。

1. **宮城県のおすすめお土産20選**
   - 宮城県の名物や特産品を使ったお土産のリストが紹介されています。和菓子や地元の名物といった商品の詳細が記載されており、どれも宮城ならではの魅力的な商品です。詳しくは[こちら](https://tohokuru.jp/blogs/feature/miyage_miyagi)を参照。

2. **地元の土産物店の一品**
   - 宮城県の地元の土産物店が提供する一品一品を紹介しています。仙台名物の牛たんや、特産のあんこを使用した和菓子など、地元ならではの味を楽しむことができます。

3. **仙台の名物食品**
   - 仙台名物の牛たんや、地元で栽培された米を使用したお菓子など、宮城の代表的なお土産が紹介されています。特に、牛たんは多くの観光客に人気のある商品です。

4. **地元の特産品を使用したお菓子のセレクション**
   - 宮城の特産九州産のいちごや、仙台藩の名の付けられたお菓子の情報が載っており、各店舗のおすすめ商品も一覧となっています。詳細は[こちら](https://amabijin.com/article/16065)でチェックできます。

これらのお土産は、宮城県を訪れた際の素敵な思い出や贈り物として、ぜひ考えてみてください。

---ご利用ありがとうございました！---
